<a href="https://colab.research.google.com/github/RyanYavari/MarineVisionFilters/blob/main/hough_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Libraries

!pip install opencv-python
!pip install opencv-python-headless
!pip install numpy
!pip install matplotlib

import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# mounting drive
from google.colab import drive
drive.mount('/content/drive')

# Load images into array
images_path = "/content/drive/Shareddrives/visionFinal/ARMSPLATES/"
images = [cv2.imread(os.path.join(images_path, f)) for f in os.listdir(images_path)]

Mounted at /content/drive


In [ ]:

# Sobel preprocessing for hough transform

def sobel_func(image):
    sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=5)
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=5)
    sobel_img = cv2.magnitude(sobel_x, sobel_y)

#Normalize and reassign the pixel values to be unsigned
    sobel_img = cv2.normalize(sobel_img, None, 0, 255, cv2.NORM_MINMAX)
    sobel_img = np.uint8(sobel_img)

#Convert sobel to grayscale
    sobel_img = cv2.cvtColor(sobel_img, cv2.COLOR_BGR2GRAY)

    return sobel_img

# Hough Transform Filter
def hough(image):
    # Apply Gaussian blur to smooth out textures and prevent unwanted circles
    gray_blurred = cv2.GaussianBlur(image, (15, 15), 2)

    # Apply Sobel edge detection to blurred image
    sobel_img = sobel_func(gray_blurred)

    # Apply HoughCircles method for screw (circle) detection
    circles = cv2.HoughCircles(
        sobel_img,
        cv2.HOUGH_GRADIENT,
        dp=1,
        minDist=20,
        param1=50,
        param2=30,
        minRadius=35, #smallest screw radius
        maxRadius=45 #biggest screw radius
    )

#Increase radius of accumulator if circle not detected
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            cv2.circle(image, (i[0], i[1]), i[2], (255, 0, 255), 10) #Magenta color, thickness = 10

# Process and save images
hough_folder = "/content/drive/Shareddrives/visionFinal/Hough_Transform_Filter_Images"
os.makedirs(hough_folder, exist_ok=True)

#Load images into hough folder
for i, img in enumerate(images):
    labeled_images = img.copy()
    hough(labeled_images)
    cv2.imwrite(os.path.join(hough_folder, f"hough_{i}.png"), labeled_images)